In [ ]:
# Copora Callosa

In [ ]:
# LDDMM landmark dynamics
import src.params as params
params.manifold = 'landmarks'

In [ ]:
from src.manifold import *
from src.metric import *

from src.plotting import *
%matplotlib inline
plt.rcParams['figure.figsize'] = 7, 7

In [ ]:
# Obs: takes time to load
from src.Regression.params import *
from src.Regression.LaplaceApprox import *

from scipy import io
A=io.loadmat('ipmi-data/matlab.mat')

# Corpus Callosum data:

In [ ]:
Seq = lambda m, n: [i*n//m + n//(2*m) for i in range(m)]

In [ ]:
n_samples.set_value(20)
N.set_value(9)
m.set_value(1)

# load data
data=io.loadmat('ipmi-data/dataM-corpora-callosa-ipmi-full.mat')
vi = data['vi'] # corpora callosa
Nobs = vi.shape[1]
N0 = vi.shape[0]/2-1
qs = vi[0:-2,:].T.reshape((Nobs,N0,2))

yObsM = qs[0:n_samples.get_value(),Seq(Nl.get_value(),N0),:]/20
yObs = yObsM.reshape((n_samples.get_value(),DIM.eval()))
print(yObs.shape)

In [ ]:
%matplotlib inline
for i in range(n_samples.get_value()):
    plt.plot(yObsM[i,:,0],yObsM[i,:,1])

In [ ]:
gen = np.empty((n_samples.get_value()), dtype = str)
ald = np.zeros(n_samples.get_value())
for i in range(n_samples.get_value()):
    ald[i] = A['combined'][0,i][2][0]
    gen[i] = A['combined'][0,i][1][0][0]

xT0 = ald.reshape((n_samples.get_value(),m.get_value()))

In [ ]:
ui0 = np.linalg.lstsq(xT0, yObs)[0].flatten()
ui0 = ui0/np.linalg.norm(ui0)

x01 = np.zeros(m.get_value())

In [ ]:
W0 = np.array([[np.linalg.norm(ui0)]])[0:m.get_value(),0:m.get_value()]
drift0 = np.array([0.01,0.01])[0:m.get_value()]

tau0 = 0.1

y0 = np.mean(yObs, axis = 0)

para0 = np.hstack((tau0,drift0,W0.flatten(),y0.flatten(),ui0.flatten()))
print("Initial para = ", para0)

# Laplace Approximation:

### Optimal processes:

In [ ]:
start = time.time()
dWtop = multprocdwt(para0,x01,yObs,xT0,n_pool=10)
diff = time.time() - start
print "time = ", diff

### Estimation:

In [ ]:
start = time.time()
print("Initial likelihood = ",loglikef(para0,x01,yObs,xT0,dWtop.flatten()))
diff = time.time() - start
print()"time = ", diff)

start = time.time()
print("Gradient = ", np.round(scipy.optimize.approx_fprime(para0,loglikef,0.001,x01,yObs,xT0,dWtop.flatten()),2)
diff = (time.time() - start)/60
print("time gradient = ", diff)

In [ ]:
paraEst = LapApprox(para0,10**(-4),300,0.2,x01,yObs,
                    xT0,dWtop.flatten())